<a href="https://colab.research.google.com/github/fatenchtioui/Proj/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

# Définir le générateur
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Définir le discriminateur
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Charger le jeu de données MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_data = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = torch.utils.data.DataLoader(mnist_data, batch_size=100, shuffle=True)

# Initialiser le générateur et le discriminateur
generator = Generator()
discriminator = Discriminator()

# Définir la fonction de perte et les optimiseurs
criterion = nn.BCELoss()
optimizer_generator = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=0.0002)

# Entraîner le GAN
num_epochs = 1000
for epoch in range(num_epochs):
    for real_data, _ in data_loader:
        batch_size = real_data.size(0)
        real_data = real_data.view(batch_size, -1)

        # Entraîner le discriminateur avec de vraies données
        real_data = Variable(real_data)
        target_real = Variable(torch.ones(batch_size, 1))
        output_real = discriminator(real_data)
        loss_real = criterion(output_real, target_real)

        # Entraîner le discriminateur avec de fausses données générées par le générateur
        noise = Variable(torch.randn(batch_size, 100))
        fake_data = generator(noise)
        target_fake = Variable(torch.zeros(batch_size, 1))
        output_fake = discriminator(fake_data.detach())  # détacher le générateur pour éviter de mettre à jour ses poids
        loss_fake = criterion(output_fake, target_fake)

        # Combinaison des pertes du discriminateur
        loss_discriminator = loss_real + loss_fake
        optimizer_discriminator.zero_grad()
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Entraîner le générateur en maximisant la probabilité que le discriminateur se trompe
        target_generator = Variable(torch.ones(batch_size, 1))
        output_generator = discriminator(fake_data)
        loss_generator = criterion(output_generator, target_generator)
        optimizer_generator.zero_grad()
        loss_generator.backward()
        optimizer_generator.step()

    # Afficher les pertes après chaque époque
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {loss_discriminator.item()}, Loss G: {loss_generator.item()}')

# Vous pouvez maintenant utiliser le générateur pour générer de nouvelles images


100%|██████████| 9912422/9912422 [00:00<00:00, 57161944.26it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 67074027.59it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 18687456.82it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11469312.92it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/1000], Loss D: 0.8756684064865112, Loss G: 1.108311414718628
Epoch [2/1000], Loss D: 1.302353858947754, Loss G: 0.7286391258239746
Epoch [3/1000], Loss D: 1.729331374168396, Loss G: 0.5930938124656677
Epoch [4/1000], Loss D: 0.8522946834564209, Loss G: 1.2757488489151
Epoch [5/1000], Loss D: 1.234795331954956, Loss G: 0.7968393564224243
Epoch [6/1000], Loss D: 1.0292267799377441, Loss G: 0.9761368632316589
Epoch [7/1000], Loss D: 0.9842821359634399, Loss G: 1.1570448875427246
Epoch [8/1000], Loss D: 0.956407904624939, Loss G: 1.1554269790649414
Epoch [9/1000], Loss D: 1.27570378780365, Loss G: 0.9117831587791443
Epoch [10/1000], Loss D: 1.3301053047180176, Loss G: 0.8544962406158447
Epoch [11/1000], Loss D: 1.1252996921539307, Loss G: 1.1433781385421753
Epoch [12/1000], Loss D: 1.226702332496643, Loss G: 0.8939675092697144
Epoch [13/1000], Loss D: 1.3510850667953491, Loss G: 0.9027860164642334
Epoch [14

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam

# Charger le jeu de données MNIST
(X_train, _), (_, _) = mnist.load_data()

# Normaliser les images
X_train = X_train / 127.5 - 1.0
X_train = np.expand_dims(X_train, axis=3)

# Paramètres
img_shape = (28, 28, 1)
latent_dim = 100

# Fonction pour construire le générateur
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    return model

# Fonction pour construire le discriminateur
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Construire et compiler le générateur
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Construire et compiler le discriminateur
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Geler le discriminateur pendant l'entraînement du GAN
discriminator.trainable = False

# Entrée pour le générateur
z = Input(shape=(latent_dim,))
img = generator(z)

# Sortie du discriminateur
validity = discriminator(img)

# Construire et compiler le modèle GAN (générateur + discriminateur)
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Fonction pour l'entraînement du GAN
def train_gan(epochs=10000, batch_size=128, save_interval=1000):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Sélectionner un lot aléatoire d'images réelles
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        # Échantillonner du bruit et générer un lot de fausses images
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        gen_imgs = generator.predict(noise)

        # Étiquettes valides pour les images réelles et fausses
        valid = np.ones((half_batch, 1))
        fake = np.zeros((half_batch, 1))

        # Entraîner le discriminateur sur les vraies images
        d_loss_real = discriminator.train_on_batch(imgs, valid)

        # Entraîner le discriminateur sur les fausses images générées
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

        # Calculer la perte totale du discriminateur
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entraîner le générateur pour tromper le discriminateur
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Afficher la progression de l'entraînement
        if epoch % save_interval == 0:
            print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

            # Enregistrer des exemples générés à intervalles réguliers
            save_generated_images(epoch)

import os

# Fonction pour enregistrer des exemples générés
def save_generated_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    # Vérifier si le répertoire existe, sinon le créer
    save_dir = "/content/drive/MyDrive/Colab Notebooks/generated_images/"
    os.makedirs(save_dir, exist_ok=True)

    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    for i in range(dim[0] * dim[1]):
        axs[i].imshow(generated_images[i, :, :, 0], cmap='gray')
        axs[i].axis('off')
    plt.tight_layout()

    # Enregistrer l'image générée
    save_path = os.path.join(save_dir, f"gan_generated_image_epoch_{epoch}.png")
    plt.savefig(save_path)
    plt.close()

# Entraîner le GAN
train_gan(epochs=10000, batch_size=128, save_interval=1000)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2/2 [==============================] - 0s 11ms/step
Epoch 6000, Discriminator Loss: 0.6997145414352417, Generator Loss: 0.8683266639709473
2/2 [==============================] - 0s 16ms/step
Epoch 7000, Discriminator Loss: 0.6488248109817505, Generator Loss: 0.8561457395553589
2/2 [==============================] - 0s 10ms/step
Epoch 8000, Discriminator Loss: 0.7091071605682373, Generator Loss: 0.8320426940917969
2/2 [==============================] - 0s 16ms/step
Epoch 9000, Discriminator Loss: 0.6560309827327728, Generator Loss: 0.8572752475738525
2/2 [==============================] - 0s 13ms/step
